# go-monkey 動作確認

[Go言語でつくるインタプリタ](https://www.oreilly.co.jp/books/9784873118222/)で実装したMonkey言語インタプリタの動作確認ノートブック

In [ ]:
// ローカルのmonkeyモジュールをGoNBの一時コンパイルディレクトリに登録
!cd "$GONB_TMP_DIR" && go mod edit -require=monkey@v0.0.0 -replace=monkey=/Users/junhat6/ghq/github.com/junhat6/go-monkey/monkey

In [ ]:
import (
    "fmt"
    "monkey/token"
    "monkey/lexer"
    "monkey/parser"
    "monkey/evaluator"
    "monkey/object"
)

// Monkeyコードを実行して結果を表示するヘルパー
func runMonkey(input string) {
    l := lexer.New(input)
    p := parser.New(l)
    program := p.ParseProgram()
    if errs := p.Errors(); len(errs) > 0 {
        fmt.Println("parse error:", errs)
        return
    }
    env := object.NewEnvironment()
    macroEnv := object.NewEnvironment()
    evaluator.DefineMacros(program, macroEnv)
    expanded := evaluator.ExpandMacros(program, macroEnv)
    result := evaluator.Eval(expanded, env)
    if result != nil {
        fmt.Println(result.Inspect())
    }
}

## Lexer（字句解析）

ソースコードをトークン列に分割する

In [ ]:
%%
l := lexer.New(`let add = fn(a, b) { a + b };`)
for tok := l.NextToken(); tok.Type != token.EOF; tok = l.NextToken() {
    fmt.Printf("%-10s %q\n", tok.Type, tok.Literal)
}

## Parser（構文解析）

トークン列からAST（抽象構文木）を生成する

In [ ]:
%%
l := lexer.New(`let x = (5 + 10) * 2;`)
p := parser.New(l)
program := p.ParseProgram()
fmt.Println(program.String())

## 式の評価

整数演算、比較、文字列結合

In [ ]:
%%
// 整数演算
runMonkey(`5 + 10 * 2`)
runMonkey(`(5 + 10) * 2`)

// 比較・真偽値
runMonkey(`1 < 2`)
runMonkey(`10 == 10`)
runMonkey(`!true`)

// 文字列結合
runMonkey(`"Hello" + " " + "World!"`)

## 変数・関数・クロージャ

In [ ]:
%%
// 変数束縛
runMonkey(`let x = 5; let y = 10; x + y;`)

// if/else式（値を返す）
runMonkey(`if (10 > 5) { "yes" } else { "no" }`)

// 関数
runMonkey(`let add = fn(a, b) { a + b }; add(3, 4);`)

// クロージャ
runMonkey(`let newAdder = fn(x) { fn(y) { x + y } }; let addTwo = newAdder(2); addTwo(5);`)

// 高階関数
runMonkey(`let apply = fn(f, x) { f(x) }; apply(fn(x) { x * 2 }, 10);`)

## データ構造と組み込み関数

In [ ]:
%%
// 配列
runMonkey(`let a = [1, 2, 3, 4, 5]; a[2];`)
runMonkey(`len([1, 2, 3])`)
runMonkey(`first([1, 2, 3])`)
runMonkey(`last([1, 2, 3])`)
runMonkey(`rest([1, 2, 3])`)
runMonkey(`push([1, 2, 3], 4)`)

// ハッシュ
runMonkey(`let h = {"name": "Monkey", "version": 1}; h["name"];`)

// 文字列の長さ
runMonkey(`len("Hello")`)

## エラーハンドリング

型の不一致などはエラーオブジェクトとして伝播する

In [ ]:
%%
runMonkey(`5 + true`)
runMonkey(`-true`)

## マクロシステム

`quote`/`unquote` でASTを操作し、`macro` で構文を拡張できる

In [ ]:
%%
// unlessマクロ: conditionがfalseのときconsequenceを評価する
runMonkey(`
let unless = macro(condition, consequence, alternative) {
    quote(if (!(unquote(condition))) {
        unquote(consequence);
    } else {
        unquote(alternative);
    });
};
unless(10 > 5, "not greater", "greater");
`)

## for式（独自拡張）

`for (init; condition; update) { body }` 構文を追加実装。
bodyの最後の評価値を返す。

In [ ]:
%%
// 基本的なforループ: 0〜4を出力
runMonkey(`
for (let i = 0; i < 5; let i = i + 1) {
    puts(i);
};
`)

// for式は最後のbody評価値を返す
runMonkey(`for (let i = 0; i < 5; let i = i + 1) { i; }`)

// 条件が最初からfalseの場合はNULL
runMonkey(`for (let i = 10; i < 5; let i = i + 1) { i; }`)

In [ ]:
%%
// 関数内でreturnと組み合わせる
runMonkey(`
let findIndex = fn(arr, target) {
    for (let i = 0; i < len(arr); let i = i + 1) {
        if (arr[i] == target) { return i; }
    };
};
findIndex([10, 20, 30, 40, 50], 30);
`)